# Create Delta table and insert 6 rows

In [1]:
from pyspark.context import SparkContext
from pyspark import SparkFiles
from pyspark.sql.session import SparkSession
from pyspark.sql.types import *

deltaSpark = (
    SparkSession.builder.master("local[1]")
    .config("spark.jars.packages", "io.delta:delta-spark_2.12:3.2.0")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .getOrCreate()
)

local_base_path = "file:/tmp/polaris/ForeignTableDemo/"

records = [
   (1, 'John', 25, 'NYC', '2023-09-28 00:00:00'),
   (2, 'Emily', 30, 'SFO', '2023-09-28 00:00:00'),
   (3, 'Michael', 35, 'ORD', '2023-09-28 00:00:00'),
   (4, 'Andrew', 40, 'NYC', '2023-10-28 00:00:00'),
   (5, 'Bob', 28, 'SEA', '2023-09-23 00:00:00'),
   (6, 'Charlie', 31, 'DFW', '2023-08-29 00:00:00')
]


schema = StructType([
   StructField("id", IntegerType(), True),
   StructField("name", StringType(), True),
   StructField("age", IntegerType(), True),
   StructField("city", StringType(), True),
   StructField("create_ts", StringType(), True)
])

df = deltaSpark.createDataFrame(records, schema)

(
   df.write
   .format("delta")
   .partitionBy("city")
   .mode("overwrite") 
   .save(f"{local_base_path}")
)


# Append 4 rows

In [2]:
records = [
   (7, 'Alice', 25, 'NYC', '2023-09-28 00:00:00'),
   (8, 'Tim', 30, 'SFO', '2023-09-28 00:00:00'),
   (9, 'Andy', 35, 'ORD', '2023-09-28 00:00:00'),
   (10, 'Bob', 40, 'SFO', '2023-10-28 00:00:00'),
]
df = deltaSpark.createDataFrame(records, schema)

(
   df.write
   .format("delta")
   .partitionBy("city")
   .mode("append") 
   .save(f"{local_base_path}")
)